In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import pyutils as ut

In [2]:
x_train = tf.convert_to_tensor(pd.read_csv("datasets/x_train.csv").to_numpy()[:, 1:])
y_train = tf.convert_to_tensor(pd.read_csv("datasets/y_train.csv").to_numpy()[:, 1:])
x_test = tf.convert_to_tensor(pd.read_csv("datasets/x_test.csv").to_numpy()[:, 1:])
y_test = tf.convert_to_tensor(pd.read_csv("datasets/y_test.csv").to_numpy()[:, 1:])

print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(70000, 26) (15000, 26) (70000, 1) (15000, 1)


In [4]:
kan_layers = [3,1]
kan_model = ut.build_model(kan_layers, x_train.shape[1], mlp=True)
kan_model.summary()

mlp_layers = [20,3,1]
mlp_model = ut.build_model(mlp_layers, x_train.shape[1], mlp=True)
mlp_model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_2 (Dense)                 │ (None, 3)              │            81 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │             4 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 85 (340.00 B)

 Trainable params: 85 (340.00 B)

 Non-trainable params: 0 (0.00 B)

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                 │ (None, 20)             │           540 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 3)              │            63 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 1)              │             4 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 607 (2.37 KB)

 Trainable params: 607 (2.37 KB)

 Non-trainable params: 0 (0.00 B)